In [1]:
# !pip install deeppavlov emoji
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
# !pip install tensorflow==2.15.1
# !pip install transformers
# !export PATH=/usr/local/cuda-12.4/bin:$PATH
# !export LD_LIBRARY_PATH=/usr/local/cuda-12.4/lib64:$LD_LIBRARY_PATH

In [2]:
!pip install transformers pandas torch spacy yake datasets

In [3]:
import os
import gdown

def download_file_if_not_exists(file_url, output_path):
    """Скачивает файл с Google Drive, если он ещё не существует в указанной директории."""
    # Проверка наличия файла
    if os.path.exists(output_path):
        print(f"Файл '{output_path}' уже существует.")
    else:
        print(f"Файл '{output_path}' не найден. Начинаю загрузку...")
        gdown.download(file_url, output_path, quiet=False)
        print(f"Файл '{output_path}' успешно загружен.")

# Указываем URL и путь к файлу
file_url = 'https://drive.google.com/uc?id=1alondqI-2IHo__mYU7KQz4Ip8ytYGHXg'
output_file_name = 'wildberries_reviews.csv'  # Укажите реальное имя файла, которое хотите сохранить
output_path = os.path.join(os.getcwd(), output_file_name)  # Полный путь к файлу

download_file_if_not_exists(file_url, output_path)


Файл '/workspace/wildberries_reviews.csv' уже существует.


In [4]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

# Загрузка необходимых ресурсов nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# !pip uninstall tensorflow tensorflow-gpu -y
# !pip uninstall torch torchvision torchaudio -y
# # Установка TensorFlow
# !pip install tensorflow==2.12.0

# # Установка PyTorch с поддержкой CUDA 12.1
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64/cuda-keyring_1.1-1_all.deb
# !dpkg -i cuda-keyring_1.1-1_all.deb
# !apt-get update
# !apt-get -y install cudnn
# !export PATH=/usr/local/cuda/bin:$PATH
# !export LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH

In [6]:
import tensorflow as tf
import torch

print(f"TensorFlow version: {tf.__version__}")
print(f"Available GPUs for TensorFlow: {tf.config.list_physical_devices('GPU')}")

print(f"PyTorch version: {torch.__version__}")
print(f"Is CUDA available for PyTorch: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version for PyTorch: {torch.version.cuda}")
    print(f"cuDNN version for PyTorch: {torch.backends.cudnn.version()}")


2024-08-02 12:19:05.758589: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-02 12:19:05.790230: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-02 12:19:05.790939: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-02 12:19:06.391997: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TensorFlow version: 2.12.0
Available GPUs for TensorFlow: []
PyTorch version: 2.4.0+cu121
Is CUDA available for PyTorch: True
CUDA version for PyTorch: 12.1
cuDNN version for PyTorch: 90100


2024-08-02 12:19:08.687290: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-02 12:19:08.687793: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [7]:
import pandas as pd
import re
import yake
from tqdm import tqdm
import nltk
from nltk.tokenize import sent_tokenize
from spacy.lang.ru.stop_words import STOP_WORDS as stopwords_ru
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from concurrent.futures import ThreadPoolExecutor
import torch
from IPython.display import display
import spacy
import time
import emoji
import os

nltk.download('punkt')
print(f"torch.cuda.is_available() - {torch.cuda.is_available()}")

nlp = spacy.load("ru_core_news_lg")

def clean_text(text):
    text = emoji.replace_emoji(text, replace='')
    return text

# Функция для предварительной обработки текста
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # Удаление лишних пробелов
    text = re.sub(r'[^\w\s]', '', text)  # Удаление знаков пунктуации
    text = text.lower()  # Приведение текста к нижнему регистру
    tokens = text.split()  # Токенизация
    tokens = [word for word in tokens if word not in stopwords_ru]  # Удаление стоп-слов
    return ' '.join(tokens)

# Настройка YAKE
kw_extractor = yake.KeywordExtractor(lan="ru", n=1, dedupLim=0.9, top=10)

def extract_aspects(text):
    keywords = kw_extractor.extract_keywords(text)
    return [kw[0] for kw in keywords]

# Загрузка модели и токенизатора для анализа тональности
print("Загрузка модели и токенизатора...")
model_name = "cointegrated/rubert-tiny-sentiment-balanced"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to('cuda' if torch.cuda.is_available() else 'cpu')
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)
print("Модель и токенизатор загружены.")

# Функция для анализа аспектов, характеристик и краткого описания
def analyze_aspects(doc):
    text = doc.text
    aspects = extract_aspects(text)
    aspect_details = {}
    for aspect in aspects:
        aspect_lemma = nlp(aspect)[0].lemma_.lower()  # Лемматизация и приведение к нижнему регистру
        sentences = [sent for sent in doc.sents if aspect in sent.text]
        if sentences:
            sentiment_result = classifier(aspect)
            aspect_sentiment = sentiment_result[0]['label']
            
            # Выбор наиболее информативного предложения
            aspect_desc = max(sentences, key=lambda sent: len([token for token in sent if token.text == aspect])).text
            
            # Выделение характеристик (прилагательных) из предложения
            characteristics = []
            for token in sentences[0]:
                if token.head.text == aspect and token.pos_ == "ADJ":
                    characteristics.append(token.text)
                elif token.text == aspect and token.dep_ in {"amod", "acomp"}:
                    characteristics.append(token.head.text)
            
            # Создание краткого описания
            description = f"{aspect_desc}. Тональность: {aspect_sentiment}. Характеристики: {', '.join(characteristics)}."
            
            if aspect_lemma not in aspect_details:
                aspect_details[aspect_lemma] = {
                    'sentiment': aspect_sentiment,
                    'description': description,
                    'characteristics': characteristics,
                    'count': 1
                }
            else:
                aspect_details[aspect_lemma]['count'] += 1
                aspect_details[aspect_lemma]['description'] += f" | {description}"
                aspect_details[aspect_lemma]['characteristics'].extend(characteristics)
    return aspect_details

def process_texts(texts, batch_size):
    """Обрабатывает список текстов, разбивая их на сегменты."""
    all_aspect_details = []
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        futures = []
        for doc in tqdm(nlp.pipe(texts, batch_size=batch_size), total=len(texts), desc="Loading texts to docs"):
            futures.append(executor.submit(analyze_aspects, doc))
        for future in tqdm(futures, total=len(texts), desc="Processing"):
            all_aspect_details.append(future.result())
    return all_aspect_details

# Загрузка данных и ограничение до 1000 отзывов
print("Загрузка данных...")
start_time = time.time()
df = pd.read_csv('wildberries_reviews.csv', nrows=1000)
df['processed_text'] = df['corrected_text'].apply(preprocess_text)
df['sentences'] = df['corrected_text'].apply(sent_tokenize)
print("Данные загружены. Время выполнения: {:.2f} секунд".format(time.time() - start_time))

# Применение функции к датасету с использованием ThreadPoolExecutor и tqdm для прогресса
print("Обработка отзывов...")
start_time = time.time()
batch_size = 128

texts = df["corrected_text"].apply(clean_text).tolist()
all_aspect_details = process_texts(texts, batch_size)

print("Обработка отзывов завершена. Время выполнения: {:.2f} секунд".format(time.time() - start_time))

# Преобразование результатов обратно в DataFrame
start_time = time.time()
df['aspect_details'] = all_aspect_details
print("Преобразование результатов завершено. Время выполнения: {:.2f} секунд".format(time.time() - start_time))

# Оценка важности аспектов для каждого товара
print("Оценка важности аспектов...")
start_time = time.time()
def evaluate_aspect_importance(df):
    aspect_counter = {}
    for _, row in df.iterrows():
        product = row['product']
        if product not in aspect_counter:
            aspect_counter[product] = {}
        for aspect, details in row['aspect_details'].items():
            if aspect not in aspect_counter[product]:
                aspect_counter[product][aspect] = {
                    'count': 0,
                    'positive': 0,
                    'negative': 0,
                    'neutral': 0,
                    'descriptions': [],
                    'characteristics': []
                }
            aspect_counter[product][aspect]['count'] += details['count']
            aspect_counter[product][aspect][details['sentiment']] += 1
            aspect_counter[product][aspect]['descriptions'].append(details['description'])
            aspect_counter[product][aspect]['characteristics'].extend(details['characteristics'])
    return aspect_counter

aspect_importance = evaluate_aspect_importance(df)
print("Оценка важности аспектов завершена. Время выполнения: {:.2f} секунд".format(time.time() - start_time))

# Создание итоговой таблицы для каждого товара
print("Создание итоговой таблицы для каждого товара...")
start_time = time.time()
product_aspects = []
for product, aspects in aspect_importance.items():
    for aspect, metrics in aspects.items():
        product_aspects.append({
            'product': product,
            'aspect': aspect,
            'count': metrics['count'],
            'positive': metrics['positive'],
            'negative': metrics['negative'],
            'neutral': metrics['neutral'],
            'descriptions': max(metrics['descriptions'], key=len),  # Выбираем наиболее информативное описание
            'characteristics': metrics['characteristics']
        })

aspect_df = pd.DataFrame(product_aspects)
aspect_df = aspect_df.sort_values(by='count', ascending=False)
print("Создание итоговой таблицы завершено. Время выполнения: {:.2f} секунд".format(time.time() - start_time))

# Вывод результатов
print("\nИтоговая таблица важных аспектов для каждого товара:")
display(aspect_df.head(10))  # Вывод первых 10 строк для примера

print("\nРезультаты анализа отзывов:")
display(df.head(10))  # Вывод первых 10 строк для примера

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


torch.cuda.is_available() - True
Загрузка модели и токенизатора...
Модель и токенизатор загружены.
Загрузка данных...
Данные загружены. Время выполнения: 0.06 секунд
Обработка отзывов...


Processing: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:55<00:00, 18.11it/s]

Обработка отзывов завершена. Время выполнения: 72.16 секунд
Преобразование результатов завершено. Время выполнения: 0.00 секунд
Оценка важности аспектов...
Оценка важности аспектов завершена. Время выполнения: 0.04 секунд
Создание итоговой таблицы для каждого товара...
Создание итоговой таблицы завершено. Время выполнения: 0.01 секунд

Итоговая таблица важных аспектов для каждого товара:


,product,aspect,count,positive,negative,neutral,descriptions,characteristics
2344,Пахнет и Точка / Ароматизатор в машину автопар...,запах,53,53,0,0,"Полная фигня, запах вкусный, но его просто нет...","[нереальный, Стойкий, Приятный, Отличный, стой..."
2734,ПолиКомПласт / Преобразователь очиститель ржав...,работать,45,0,0,45,Реально штука работает намазать на ржавчину со...,[лучше]
2738,ПолиКомПласт / Преобразователь очиститель ржав...,ржавчина,38,0,0,36,Реально штука работает намазать на ржавчину со...,"[небольшая, глубокую, рыхлой, свежей, глубокая..."
2341,Пахнет и Точка / Ароматизатор в машину автопар...,аромат,36,0,0,35,"Первый день, аромат есть, пахнет очень вкусно,...","[вкусный, Классный, любимый, Отличный, Шикарны..."
2773,ПолиКомПласт / Преобразователь очиститель ржав...,отличный,33,33,0,0,"Отличная паста, ни чем не хуже хваленой КППС, ...","[Отлично, средство, преобразователь, мазеобраз..."
2350,Пахнет и Точка / Ароматизатор в машину автопар...,пахнуть,26,0,3,23,"Во-первых пришел совсем другой аромат, во втор...","[третьих, вкусно, навязчиво, приятная, нужно]"
2787,ПолиКомПласт / Преобразователь очиститель ржав...,средство,22,0,0,21,"Думаю слова тут лишние, супер средство, не ожи...","[хорошее, лишние, Отличное, Хорошее, хорошее, ..."
1901,ВПМ / Антибукс - антипробуксовочные траки утол...,помочь,20,18,0,2,"Не во всех случаях, конечно, эти Анти буксы мо...",[]
2799,ПолиКомПласт / Преобразователь очиститель ржав...,рекомендовать,19,19,0,0,"Удивил, удаляет ржавчину на глазах до металла,...",[]
1543,Hangkai / Лебедка электрическая влагозащитная ...,лебёдка,18,0,0,18,Коробка с лебедкой пришла как будто её собаки ...,"[первая, Лебедка, Отличная, вторая, хорошая, Х..."



Результаты анализа отзывов:


,Unnamed: 0,review_full_text,review_rating,product,category,url,corrected_text,processed_text,sentences,aspect_details
0,0,Работает хорошо.,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Работает хорошо.,работает,[Работает хорошо.],"{'работать': {'sentiment': 'neutral', 'descrip..."
1,1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...",пришло быстро целое вид завтра испытывать,"[Пришло быстро, все целое на вид., Завтра буду...","{'прийти': {'sentiment': 'neutral', 'descripti..."
2,2,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...",купил квадр поднятия отвала установка заняла час,"[Купил на квадр для поднятия отвала, установка...","{'купил': {'sentiment': 'neutral', 'descriptio..."
3,3,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...,лебёдка хорошая инструкции ни слова сборку кре...,"[Лебёдка хорошая., Но в инструкции ни слова пр...","{'лебёдка': {'sentiment': 'neutral', 'descript..."
4,4,"Всё в комплекте, есть инструкция на русском яз...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Всё в комплекте, есть инструкция на русском яз...",комплекте инструкция русском языке,"[Всё в комплекте, есть инструкция на русском я...","{'комплект': {'sentiment': 'neutral', 'descrip..."
5,5,Муж ещё не пробовал,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Муж ещё не пробовал,муж пробовал,[Муж ещё не пробовал],"{'муж': {'sentiment': 'neutral', 'description'..."
6,6,👍,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,👍,,[👍],{}
7,7,"Пока при проверке работает нормально, а там по...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пока при проверке работает нормально, а там по...",проверке работает нормально посмотрим хватит,"[Пока при проверке работает нормально, а там п...","{'нормальный': {'sentiment': 'positive', 'desc..."
8,8,"Не комплект, нет крюка, заметил поздно, кроме ...",3,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Не комплект, нет крюка, заметил поздно, кроме ...",комплект крюка заметил поздно возврата предлож...,"[Не комплект, нет крюка, заметил поздно, кроме...","{'комплект': {'sentiment': 'neutral', 'descrip..."
9,9,"При первой поездки в лес, барабан отломился ко...",1,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"При первой поездки в лес, барабан отломился ко...",первой поездки лес барабан отломился начал нам...,"[При первой поездки в лес, барабан отломился к...","{'лес': {'sentiment': 'neutral', 'description'..."


In [8]:
import pandas as pd
from collections import defaultdict

# Создание итоговой таблицы для каждого товара
def create_aspect_summary(df):
    aspect_counter = defaultdict(lambda: defaultdict(lambda: {
        'count': 0,
        'positive': 0,
        'negative': 0,
        'neutral': 0,
        'descriptions': [],
        'characteristics': []
    }))
    
    for _, row in df.iterrows():
        product = row['product']
        for aspect, details in row['aspect_details'].items():
            aspect_counter[product][aspect]['count'] += details['count']
            aspect_counter[product][aspect][details['sentiment']] += 1
            aspect_counter[product][aspect]['descriptions'].append(details['description'])
            aspect_counter[product][aspect]['characteristics'].extend(details['characteristics'])
    
    return aspect_counter

def generate_aspect_df(aspect_counter):
    product_aspects = []
    for product, aspects in aspect_counter.items():
        for aspect, metrics in aspects.items():
            product_aspects.append({
                'product': product,
                'aspect': aspect,
                'count': metrics['count'],
                'positive': metrics['positive'],
                'negative': metrics['negative'],
                'neutral': metrics['neutral'],
                'descriptions': max(metrics['descriptions'], key=len),  # Выбираем наиболее информативное описание
                'characteristics': metrics['characteristics']
            })
    return pd.DataFrame(product_aspects).sort_values(by='count', ascending=False)

# Оценка важности аспектов
print("Оценка важности аспектов...")
aspect_counter = create_aspect_summary(df)
aspect_df = generate_aspect_df(aspect_counter)
print("Оценка важности аспектов завершена.")

# Функции для рекомендаций по улучшению продуктов
def improve_products(aspect_df):
    improvements = []
    for _, row in aspect_df.iterrows():
        product = row['product']
        aspect = row['aspect']
        if row['negative'] > row['positive']:
            improvements.append({
                'product': product,
                'aspect': aspect,
                'action': 'improve',
                'description': row['descriptions'],
                'characteristics': row['characteristics']
            })
        else:
            improvements.append({
                'product': product,
                'aspect': aspect,
                'action': 'highlight',
                'description': row['descriptions'],
                'characteristics': row['characteristics']
            })
    return pd.DataFrame(improvements)

# Реализация рекомендаций
print("Реализация рекомендаций...")
improvement_df = improve_products(aspect_df)
print("Рекомендации по улучшению продуктов:")
display(improvement_df)

# Вывод улучшений
print("\nРекомендации для улучшения продуктов:")
for _, row in improvement_df.iterrows():
    if row['action'] == 'improve':
        print(f"Товар: {row['product']}")
        print(f"Аспект для улучшения: {row['aspect']}")
        print(f"Описание: {row['description']}")
        print(f"Характеристики: {', '.join(row['characteristics'])}")
        print(f"Рекомендация: Улучшить {row['aspect']} на основе обратной связи.")
        print("="*50)
    else:
        print(f"Товар: {row['product']}")
        print(f"Аспект для продвижения: {row['aspect']}")
        print(f"Описание: {row['description']}")
        print(f"Характеристики: {', '.join(row['characteristics'])}")
        print(f"Рекомендация: Подчеркнуть {row['aspect']} в маркетинговых материалах.")
        print("="*50)


Оценка важности аспектов...
Оценка важности аспектов завершена.
Реализация рекомендаций...
Рекомендации по улучшению продуктов:


,product,aspect,action,description,characteristics
0,Пахнет и Точка / Ароматизатор в машину автопар...,запах,highlight,"Полная фигня, запах вкусный, но его просто нет...","[нереальный, Стойкий, Приятный, Отличный, стой..."
1,ПолиКомПласт / Преобразователь очиститель ржав...,работать,highlight,Реально штука работает намазать на ржавчину со...,[лучше]
2,ПолиКомПласт / Преобразователь очиститель ржав...,ржавчина,highlight,Реально штука работает намазать на ржавчину со...,"[небольшая, глубокую, рыхлой, свежей, глубокая..."
3,Пахнет и Точка / Ароматизатор в машину автопар...,аромат,highlight,"Первый день, аромат есть, пахнет очень вкусно,...","[вкусный, Классный, любимый, Отличный, Шикарны..."
4,ПолиКомПласт / Преобразователь очиститель ржав...,отличный,highlight,"Отличная паста, ни чем не хуже хваленой КППС, ...","[Отлично, средство, преобразователь, мазеобраз..."
...,...,...,...,...,...
3200,ПК ЛИМ / Браслеты цепи противоскольжения,что,highlight,"Я уже эксплуатирую похожие браслеты, поэтому з...",[]
3201,ПК ЛИМ / Браслеты цепи противоскольжения,похожий,highlight,"Я уже эксплуатирую похожие браслеты, поэтому з...",[браслеты]
3202,ПК ЛИМ / Браслеты цепи противоскольжения,гололёд,highlight,Мне помогли тронуться в гололёд. Тональность: ...,[]
3203,ПК ЛИМ / Браслеты цепи противоскольжения,тронуться,highlight,Мне помогли тронуться в гололёд. Тональность: ...,[]



Рекомендации для улучшения продуктов:
Товар: Пахнет и Точка / Ароматизатор в машину автопарфюм подвесной
Аспект для продвижения: запах
Описание: Полная фигня, запах вкусный, но его просто нет поесть переворачиваешь флакон все мокнет, но запах едва уловим, брал и ранее такие, (поляронная вишня и шанель)такой проблемы не было запах был яркий и стойкий, тут хоть флакон в нос засунь едва уловимый тусклый запах если вы не знаете на что он похож то я часто путаю толи деревянный (лимит вариантов исчерпан) пахнет толи парфюм в общем запах (лимит вариантов исчерпан) древесины.. Тональность: positive. Характеристики: тусклый, общем.
Характеристики: нереальный, Стойкий, Приятный, Отличный, стойкий, красивый, Хороший, тусклый, общем, обалденный, Неплохой, Хороший, Отличный, нормальный, Отличный, Крутые, огонь
Рекомендация: Подчеркнуть запах в маркетинговых материалах.
Товар: ПолиКомПласт / Преобразователь очиститель ржавчины авто металл антикор
Аспект для продвижения: работать
Описание: Реально ш